# Lab 3-2: 모니터링 시스템 대화형 실습

Jupyter Notebook을 통해 Prometheus 메트릭과 모니터링 시스템을 대화형으로 학습합니다.

## 📋 실습 내용

1. 환경 설정 및 Prometheus 연결
2. Prometheus 메트릭 타입 (Counter, Gauge, Histogram, Summary)
3. Custom Metrics 생성 (중복 방지)
4. PromQL 쿼리 실습
5. 메트릭 시각화
6. A/B 테스트 시뮬레이션
7. 알림 규칙 테스트

## 🏢 환경

- **Kubeflow Jupyter Notebook** (Kubernetes 클러스터 내부)
- **Prometheus**: `http://prometheus.monitoring.svc.cluster.local:9090`
- **Namespace**: `monitoring`

---

## 1. 환경 설정 및 패키지 설치

In [ ]:
# 필요한 패키지 설치
!pip install prometheus-client requests pandas numpy matplotlib seaborn -q

print("✅ 패키지 설치 완료!")

In [ ]:
# Import 필요한 라이브러리
import time
import random
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from prometheus_client import REGISTRY, Counter, Gauge, Histogram, Summary, start_http_server
from datetime import datetime, timedelta
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

# Plot 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Import 완료")

---

## 2. Prometheus 연결 설정 (자동 환경 감지)

In [ ]:
# ============================================================
# Prometheus 연결 설정 - 환경 자동 감지
# ============================================================

# 환경 감지
is_kubernetes = os.path.exists('/var/run/secrets/kubernetes.io')

# Prometheus URL 설정
if is_kubernetes:
    # Kubeflow Notebook: Kubernetes Service DNS 사용
    PROMETHEUS_URL = os.getenv(
        'PROMETHEUS_URL',
        'http://prometheus.monitoring.svc.cluster.local:9090'
    )
    ENV = "Kubeflow Notebook (Kubernetes 클러스터 내부)"
else:
    # 로컬: Port-forward 사용
    PROMETHEUS_URL = os.getenv('PROMETHEUS_URL', 'http://localhost:9090')
    ENV = "로컬 개발 환경 (Port-forward 필요)"

print("=" * 70)
print(" " * 20 + "Prometheus 연결 설정")
print("=" * 70)
print(f"\n🏢 환경: {ENV}")
print(f"📡 Prometheus URL: {PROMETHEUS_URL}")
print(f"📍 Namespace: monitoring")
print(f"🔌 Service: prometheus")
print(f"🔢 Port: 9090")

# Prometheus 쿼리 함수
def query_prometheus(query, timeout=10):
    """Prometheus 쿼리 실행"""
    try:
        response = requests.get(
            f"{PROMETHEUS_URL}/api/v1/query",
            params={'query': query},
            timeout=timeout
        )
        if response.status_code == 200:
            return response.json()
        else:
            return {'error': f'HTTP {response.status_code}: {response.text}'}
    except requests.exceptions.ConnectionError as e:
        return {
            'error': 'Connection failed',
            'hint': 'Prometheus 서비스가 monitoring namespace에 배포되어 있는지 확인하세요'
        }
    except requests.exceptions.Timeout:
        return {
            'error': 'Request timeout',
            'hint': 'Prometheus 서비스가 응답하지 않습니다'
        }
    except Exception as e:
        return {'error': str(e)}

# 연결 테스트
print("\n" + "=" * 70)
print(" " * 25 + "연결 테스트")
print("=" * 70)

try:
    response = requests.get(
        f"{PROMETHEUS_URL}/api/v1/query",
        params={'query': 'up'},
        timeout=5
    )
    
    if response.status_code == 200:
        print("\n✅ Prometheus 연결 성공!")
        
        data = response.json()
        if data.get('status') == 'success':
            results = data.get('data', {}).get('result', [])
            print(f"✅ {len(results)}개 타겟 발견\n")
            
            # 타겟 정보 출력
            print("타겟 목록:")
            for i, result in enumerate(results[:10], 1):
                metric = result.get('metric', {})
                value = result.get('value', [None, 'N/A'])[1]
                job = metric.get('job', 'unknown')
                instance = metric.get('instance', 'unknown')
                status = "🟢 UP" if value == '1' else "🔴 DOWN"
                print(f"   {i}. [{status}] {job:20s} @ {instance}")
        
        print("\n" + "=" * 70)
        print("✅ 설정 완료! 이제 메트릭 실습을 시작할 수 있습니다")
        print("=" * 70)
        
    else:
        print(f"\n❌ 연결 실패: HTTP {response.status_code}")
        print(f"   Response: {response.text[:200]}")
        if not is_kubernetes:
            print("\n💡 해결 방법:")
            print("   새 터미널에서 실행: kubectl port-forward -n monitoring svc/prometheus 9090:9090")
        
except requests.exceptions.ConnectionError as e:
    print(f"\n❌ 연결 실패: {e}")
    print("\n💡 해결 방법:")
    if is_kubernetes:
        print("   kubectl get svc -n monitoring")
        print("   kubectl get pods -n monitoring")
    else:
        print("   kubectl port-forward -n monitoring svc/prometheus 9090:9090")
        
except Exception as e:
    print(f"\n❌ 에러: {e}")

---

## 3. Prometheus 메트릭 생성 (중복 방지)

In [ ]:
# ============================================================
# 메트릭 헬퍼 함수 - 중복 방지
# ============================================================

def get_or_create_metric(metric_class, name, documentation, labelnames, **kwargs):
    """
    메트릭 가져오기 또는 생성 (중복 방지)
    
    이 함수는 Jupyter Notebook에서 셀을 여러 번 실행해도
    'Duplicated timeseries' 에러가 발생하지 않도록 합니다.
    """
    # 이미 존재하는지 확인
    for collector in list(REGISTRY._collector_to_names.keys()):
        if hasattr(collector, '_name') and collector._name == name:
            return collector
    
    # 없으면 새로 생성
    try:
        if kwargs:
            return metric_class(name, documentation, labelnames, **kwargs)
        else:
            return metric_class(name, documentation, labelnames)
    except ValueError as e:
        # 레지스트리 충돌 발생 시 제거 후 재생성
        if name in REGISTRY._names_to_collectors:
            try:
                REGISTRY.unregister(REGISTRY._names_to_collectors[name])
            except:
                pass
        
        if kwargs:
            return metric_class(name, documentation, labelnames, **kwargs)
        else:
            return metric_class(name, documentation, labelnames)

print("=" * 70)
print("✅ 메트릭 헬퍼 함수 준비 완료")
print("=" * 70)
print("\n이제 Jupyter 셀을 여러 번 실행해도 에러가 발생하지 않습니다!")
print("\n사용 예:")
print("  counter = get_or_create_metric(Counter, 'name', 'doc', ['labels'])")
print("  gauge = get_or_create_metric(Gauge, 'name', 'doc', ['labels'])")
print("  histogram = get_or_create_metric(Histogram, 'name', 'doc', ['labels'], buckets=(...))")

---

## 4. Counter (카운터) 실습

In [ ]:
# Counter 생성 (중복 방지)
prediction_counter = get_or_create_metric(
    Counter,
    'model_predictions_total',
    'Total number of predictions',
    ['model_version', 'status']
)

print("=" * 70)
print(" " * 25 + "Counter 실습")
print("=" * 70)
print("\nCounter: 단조 증가하는 값 (0에서 시작, 증가만 가능)")
print("예: 요청 수, 에러 수, 완료된 작업 수\n")

# Counter 증가
print("10개 예측 시뮬레이션...")
for i in range(10):
    if random.random() > 0.1:  # 90% 성공
        prediction_counter.labels(model_version='v1.0', status='success').inc()
        print(f"  {i+1}. ✅ 예측 성공")
    else:  # 10% 실패
        prediction_counter.labels(model_version='v1.0', status='error').inc()
        print(f"  {i+1}. ❌ 예측 실패")
    time.sleep(0.1)

print("\n✅ Counter 증가 완료!")
print("💡 현재 값은 다음 셀의 HTTP Server를 시작한 후 /metrics에서 확인 가능합니다")

---

## 5. Gauge (게이지) 실습

In [ ]:
# Gauge 생성 (중복 방지)
mae_gauge = get_or_create_metric(
    Gauge,
    'model_mae_score',
    'Current MAE score of the model',
    ['model_version']
)

r2_gauge = get_or_create_metric(
    Gauge,
    'model_r2_score',
    'Current R² score of the model',
    ['model_version']
)

print("=" * 70)
print(" " * 20 + "Gauge 실습 - 모델 성능 메트릭")
print("=" * 70)
print("\nGauge: 증가/감소 모두 가능한 값")
print("예: CPU 사용률, 메모리 사용량, 모델 성능 점수\n")

# Gauge 값 시뮬레이션
mae_values = []
r2_values = []

print("20스텝 동안 모델 성능 변화 시뮬레이션...\n")
for i in range(20):
    # v1.0 모델 - 안정적
    mae_v1 = 0.445 + random.gauss(0, 0.01)
    r2_v1 = 0.798 + random.gauss(0, 0.005)
    
    # v2.0 모델 - 더 좋은 성능
    mae_v2 = 0.362 + random.gauss(0, 0.01)
    r2_v2 = 0.885 + random.gauss(0, 0.005)
    
    # Gauge 업데이트
    mae_gauge.labels(model_version='v1.0').set(mae_v1)
    r2_gauge.labels(model_version='v1.0').set(r2_v1)
    mae_gauge.labels(model_version='v2.0').set(mae_v2)
    r2_gauge.labels(model_version='v2.0').set(r2_v2)
    
    mae_values.append({'v1.0': mae_v1, 'v2.0': mae_v2})
    r2_values.append({'v1.0': r2_v1, 'v2.0': r2_v2})
    
    if (i + 1) % 5 == 0:
        print(f"Step {i+1:2d} | v1.0: MAE={mae_v1:.4f}, R²={r2_v1:.4f} | v2.0: MAE={mae_v2:.4f}, R²={r2_v2:.4f}")
    
    time.sleep(0.05)

print("\n✅ Gauge 업데이트 완료!")
print(f"\n📊 평균 성능:")
print(f"   v1.0: MAE={np.mean([v['v1.0'] for v in mae_values]):.4f}, R²={np.mean([v['v1.0'] for v in r2_values]):.4f}")
print(f"   v2.0: MAE={np.mean([v['v2.0'] for v in mae_values]):.4f}, R²={np.mean([v['v2.0'] for v in r2_values]):.4f}")

---

## 6. Histogram (히스토그램) 실습

In [ ]:
# Histogram 생성 (중복 방지)
latency_histogram = get_or_create_metric(
    Histogram,
    'model_prediction_latency_seconds',
    'Prediction latency in seconds',
    ['model_version'],
    buckets=(0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 1.0, 2.5, 5.0)
)

print("=" * 70)
print(" " * 20 + "Histogram 실습 - 예측 레이턴시")
print("=" * 70)
print("\nHistogram: 값의 분포를 측정 (버킷별 빈도)")
print("예: 응답 시간, 요청 크기\n")

# Latency 시뮬레이션
latencies_v1 = []
latencies_v2 = []

print("100개 예측의 레이턴시 측정...")
for i in range(100):
    # v1.0: 평균 50ms, 더 큰 변동
    latency_v1 = abs(random.gauss(0.05, 0.02))
    latency_histogram.labels(model_version='v1.0').observe(latency_v1)
    latencies_v1.append(latency_v1)
    
    # v2.0: 평균 40ms, 더 안정적
    latency_v2 = abs(random.gauss(0.04, 0.01))
    latency_histogram.labels(model_version='v2.0').observe(latency_v2)
    latencies_v2.append(latency_v2)

print(f"\n✅ Histogram 관측 완료!")
print(f"\n📊 v1.0 레이턴시: 평균 {np.mean(latencies_v1)*1000:.2f}ms, p95 {np.percentile(latencies_v1, 95)*1000:.2f}ms, p99 {np.percentile(latencies_v1, 99)*1000:.2f}ms")
print(f"📊 v2.0 레이턴시: 평균 {np.mean(latencies_v2)*1000:.2f}ms, p95 {np.percentile(latencies_v2, 95)*1000:.2f}ms, p99 {np.percentile(latencies_v2, 99)*1000:.2f}ms")

---

## 7. Prometheus HTTP Server 시작

In [ ]:
# Prometheus HTTP Server 시작
PORT = 8000

print("=" * 70)
print(" " * 20 + "Prometheus Metrics Server")
print("=" * 70)

try:
    start_http_server(PORT)
    print(f"\n✅ Metrics Server 시작 성공!")
    print(f"\n📡 Metrics Endpoint: http://localhost:{PORT}/metrics")
    print(f"\n새 터미널에서 확인:")
    print(f"  curl http://localhost:{PORT}/metrics | grep model")
    print(f"\n또는 브라우저에서 접속하세요!")
except OSError as e:
    print(f"\n⚠️  포트 {PORT}이(가) 이미 사용 중입니다")
    print("   Metrics server가 이미 실행 중일 수 있습니다")
    print(f"   계속 진행 가능: http://localhost:{PORT}/metrics")

---

## 8. 메트릭 시각화

In [ ]:
# MAE & R² 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# MAE 비교
mae_df = pd.DataFrame(mae_values)
mae_df.plot(ax=ax1, title='Model MAE Over Time', ylabel='MAE', xlabel='Step', linewidth=2)
ax1.legend(['v1.0', 'v2.0'], loc='best')
ax1.grid(True, alpha=0.3)

# R² 비교
r2_df = pd.DataFrame(r2_values)
r2_df.plot(ax=ax2, title='Model R² Score Over Time', ylabel='R² Score', xlabel='Step', linewidth=2)
ax2.legend(['v1.0', 'v2.0'], loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 모델 성능 비교:")
mae_improvement = (1 - mae_df['v2.0'].mean() / mae_df['v1.0'].mean()) * 100
print(f"   v1.0: MAE={mae_df['v1.0'].mean():.4f}, R²={r2_df['v1.0'].mean():.4f}")
print(f"   v2.0: MAE={mae_df['v2.0'].mean():.4f}, R²={r2_df['v2.0'].mean():.4f}")
print(f"\n✨ v2.0이 MAE {mae_improvement:.1f}% 개선!")

In [ ]:
# Latency 분포 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1.hist(np.array(latencies_v1)*1000, bins=30, alpha=0.7, label='v1.0', color='blue')
ax1.hist(np.array(latencies_v2)*1000, bins=30, alpha=0.7, label='v2.0', color='orange')
ax1.set_xlabel('Latency (ms)')
ax1.set_ylabel('Frequency')
ax1.set_title('Prediction Latency Distribution')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Box plot
latency_df = pd.DataFrame({
    'v1.0': np.array(latencies_v1)*1000,
    'v2.0': np.array(latencies_v2)*1000
})
latency_df.boxplot(ax=ax2)
ax2.set_ylabel('Latency (ms)')
ax2.set_title('Latency Comparison')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 9. PromQL 쿼리 실습

In [ ]:
# PromQL 쿼리 예제
queries = [
    ("현재 MAE 값", 'model_mae_score'),
    ("현재 R² 값", 'model_r2_score'),
    ("총 예측 수", 'model_predictions_total'),
    ("최근 5분 예측 증가율", 'rate(model_predictions_total[5m])'),
    ("p95 레이턴시", 'histogram_quantile(0.95, rate(model_prediction_latency_seconds_bucket[5m]))'),
]

print("=" * 70)
print(" " * 25 + "PromQL 쿼리 예제")
print("=" * 70)

# Prometheus 연결 확인
connection_ok = False
try:
    test = requests.get(f"{PROMETHEUS_URL}/api/v1/query", params={'query': 'up'}, timeout=2)
    connection_ok = (test.status_code == 200)
except:
    pass

if not connection_ok:
    print("\n⚠️  Prometheus에 연결할 수 없습니다!")
    print(f"   현재 URL: {PROMETHEUS_URL}")
    print("\n💡 해결 방법:")
    if not is_kubernetes:
        print("   kubectl port-forward -n monitoring svc/prometheus 9090:9090")
    else:
        print("   kubectl get svc -n monitoring")
        print("   kubectl get pods -n monitoring")
else:
    print("\n✅ Prometheus 연결 확인됨\n")
    
    for description, query in queries:
        print(f"{'='*70}")
        print(f"📊 {description}")
        print(f"   Query: {query}")
        
        result = query_prometheus(query)
        
        if 'error' in result:
            print(f"   ⚠️  Error: {result['error']}")
            if 'hint' in result:
                print(f"   💡 Hint: {result['hint']}")
        elif result.get('status') == 'success':
            data = result.get('data', {}).get('result', [])
            if data:
                for item in data[:3]:
                    labels = item.get('metric', {})
                    value = item.get('value', [None, 'N/A'])[1]
                    label_str = ', '.join([f"{k}={v}" for k, v in labels.items()]) if labels else 'no labels'
                    print(f"   ✅ [{label_str}] = {value}")
            else:
                print("   ℹ️  No data (메트릭이 Prometheus에 수집되지 않았을 수 있습니다)")
        print()

---

## 10. 실습 완료!

### ✅ 학습한 내용

- Prometheus 연결 설정 (환경 자동 감지)
- 메트릭 중복 방지 (Jupyter 재실행 가능)
- Prometheus 메트릭 타입 (Counter, Gauge, Histogram)
- PromQL 쿼리 작성 및 실행
- 메트릭 시각화 (Matplotlib)

### 📚 다음 단계

1. **Grafana 대시보드** - 메트릭을 실시간으로 시각화
2. **Alertmanager 설정** - 알림 규칙 구성 및 Slack 연동
3. **프로덕션 배포** - Kubernetes에 전체 모니터링 스택 배포

---

© 2024 현대오토에버 MLOps Training - Lab 3-2  
**Environment**: Kubeflow Jupyter Notebook  
**Prometheus**: http://prometheus.monitoring.svc.cluster.local:9090